In [1]:
from __future__ import print_function
from ROOT import TFile
import ROOT
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
from math import sqrt,cos,acos,pi,exp,sin,atan2,log
from numpy import arange,mean,asarray,isnan,isinf
import pickle
from scipy import stats

from numpy import mean,std,asarray,vstack
from scipy.stats import gaussian_kde
import csv
import sys
import pandas as pd
import os

from SelectionHelper import BDTensemble,selection

fset = 12 # 0 = open data, 1-7 = fake datasets, 8 = High E far sideband, 9 = Low BDT far sideband, 10 = Final Sideband Process 1, 11 = Final Sideband Process 2
cutMode = 0 # 0 = Final Selection, 1 = Kin Cut Sample, 2 = High E, 3 = Low BDT
if fset==8: cutMode = 2
if fset==9: cutMode = 3
    
savedir = {0:'FinalSelection',1:'KinCut',2:'HighE',3:'LowBDT'}[cutMode]

# POT Breakdown for Each Run
C1_POT = 1.558e+20 + 1.129e+17 + 1.869e+19
D2_POT = 1.63e+20 + 2.964e+19 + 1.239e+19
E1_POT = 5.923e+19
F1_POT = 4.3e+19
G1_POT = 1.701e+20 + 2.97e+19 + 1.524e+17



inDat1 = None
inDat2 = None
inDat3 = None

if fset==0: 
    inDat1 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v28_wctagger_5e19.root")
    mpid_rerun_dat_1 = np.empty(0)
    inDat3 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v28_wctagger_run3_bnb1e19.root")
    mpid_rerun_dat_3 = np.empty(0)
    DAT_POT1 = 4.403e19
    DAT_POT2 = 0
    DAT_POT3 = 8.786e+18
elif fset==1: 
    inDat1 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_fakedata_set1_run1_dlana.root")
    inDat3 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_fakedata_set1_run3b_dlana.root")
    DAT_POT1  = 1.935765e20
    DAT_POT2 = 0
    DAT_POT3  = 2.99974063e20
elif fset==2: 
    inDat1 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_fakedata_set2_run1_dlana.root")
    inDat3 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_fakedata_set2_run3b_dlana.root")
    DAT_POT1 = 4.0129213616e20
    DAT_POT2 = 0
    DAT_POT3 = 3.85169302616e20
elif fset==3: 
    inDat1 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_fakedata_set3_run1_dlana.root")
    inDat3 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_fakedata_set3_run3b_dlana.root")
    DAT_POT1 = 4.024614415411e20
    DAT_POT2 = 0
    DAT_POT3 = 3.9685728933181e20
elif fset==4: 
    inDat1 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_fakedata_set4_run1_dlana.root")
    inDat3 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_fakedata_set4_run3b_dlana.root")
    DAT_POT1 = 3.907905653e20
    DAT_POT2 = 0
    DAT_POT3 = 3.92356759e20
elif fset==5: 
    inDat1 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_fakedata_set5_run1_dlana.root")
    inDat3 = None
    DAT_POT1 = 7.00587e20
    DAT_POT2 = 0
    DAT_POT3 = 0
elif fset==7: 
    inDat1 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_fakedata_set7_run1_dlana.root")
    inDat3 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_fakedata_set7_run3_dlana.root")
    DAT_POT1 = 1.8382999360024655e+20
    DAT_POT2 = 0
    DAT_POT3 = 2.05E+20
elif(fset==8):
    inDat1 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/highE_total_C1_D2_E1_F1_G1.root")
    inDat3 = None
    mpid_rerun_dat_3 = None
    DAT_POT1 = C1_POT
    DAT_POT2 = D2_POT+E1_POT
    DAT_POT3 = F1_POT+G1_POT
    DAT_POT1 = DAT_POT1 + DAT_POT2 + DAT_POT3
elif(fset==9):
    inDat1 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/lowBDT_total_C1_D2_E1_G1.root")
    inDat3 = None
    mpid_rerun_dat_3 = None
    DAT_POT1 = C1_POT
    DAT_POT2 = D2_POT+E1_POT
    DAT_POT3 = G1_POT
    DAT_POT1 = DAT_POT1 + DAT_POT2 + DAT_POT3
elif(fset==10):
    inDat1 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/nu1e1p_near_and_far_sideband_20210623.root")
    inDat3 = None
    mpid_rerun_dat_3 = None
    DAT_POT1 = 1.631e+20
    DAT_POT2 = 2.750e+20
    DAT_POT3 = 2.291e+20
    DAT_POT1 = DAT_POT1 + DAT_POT2 + DAT_POT3
elif(fset==11):
    inDat1 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v29e_dl_run1_nearbox_fvv_20210721.root")
    inDat2 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v29e_dl_run2_nearbox_fvv_20210721.root")
    inDat3 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v29e_dl_run3_nu1e1p_near_sideband_dlana_v1_1_4_fvv_20210715.root")
    mpid_rerun_dat_3 = None
    DAT_POT1 = 1.69E+20
    DAT_POT2 = 2.73E+20
    DAT_POT3 = 2.44E+20
elif(fset==12):
    inDat1 = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/BoxOpening/run123_1e1p_signal_near_far_fvv_2.root")
    inDat3 = None
    mpid_rerun_dat_3 = None
    DAT_POT1 = 1.631e+20
    DAT_POT2 = 2.750e+20
    DAT_POT3 = 2.291e+20
    DAT_POT1 = DAT_POT1 + DAT_POT2 + DAT_POT3

# if cutMode==0 and fset==0:
#     inDat1 = None
#     inDat3 = None
#     DAT_POT1 = C1_POT
#     DAT_POT2 = E1_POT+D2_POT
#     DAT_POT3 = F1_POT+G1_POT
    
DATA_POT_TOT = DAT_POT1 + DAT_POT2 + DAT_POT3

#Jarrett's numbers
#DAT_POT1 = 1.68e+20
#DAT_POT3 = 5.28e+20

#DAT_POT1 = 1e20
#DAT_POT2 = 1e20
#DAT_POT3 = 1e20

print(DAT_POT1,DAT_POT2,DAT_POT3)
print(DAT_POT1+DAT_POT2+DAT_POT3)
POTDICT = {1:DAT_POT1,2:DAT_POT2,3:DAT_POT3}


if inDat1 is not None: 
    if fset==11: inDatFF1 = inDat1.Get("FinalVertexVariables")
    else: inDatFF1 = inDat1.Get("dlana/FinalVertexVariables")
else: inDatFF1 = None
if inDat2 is not None:
    if fset==11: inDatFF2 = inDat2.Get("FinalVertexVariables")
    else: inDatFF2 = inDat2.Get("dlana/FinalVertexVariables")
else: inDatFF2 = None
if inDat3 is not None: inDatFF3 = inDat3.Get("dlana/FinalVertexVariables")
else: inDatFF3 = None

# if(fset in [1,2,3]):
#     fset_trueLabel = open('fset'+str(fset)+'_truthLabels.txt','r').read().split()
#     fset_label_dict = {'run':[],'subrun':[],'event':[],'top_label':[],'DL_label':[]}

#     for i in range(len(fset_trueLabel)/5):
#         fset_label_dict['run'].append(int(fset_trueLabel[5*i]))
#         fset_label_dict['subrun'].append(int(fset_trueLabel[5*i+1]))
#         fset_label_dict['event'].append(int(fset_trueLabel[5*i+2]))
#         fset_label_dict['top_label'].append(fset_trueLabel[5*i+3])
#         fset_label_dict['DL_label'].append(fset_trueLabel[5*i+4])
# else:
#     fset_trueLabel = None



Welcome to JupyROOT 6.16/00
6.672e+20 2.75e+20 2.291e+20
1.1713e+21


In [2]:
# ---------------- Run 1 -------------- #

inExt1      = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v28_wctagger_extbnb_finalbdt.root")
inOvr1      = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/run1_overlay_stripmine.root")
inNue1      = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/run1_intrinsics_rerun_stripmine.root")
inNueLow1   = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v29e_run1_bnb_intrinsic_nue_overlay_LowE.root")
inNumuLow1  = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v29e_run1_bnb_nu_overlay_LowE.root")
inNCPi01    = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v40_NCPi0_run1_dlana_stripped_v2.root")
inCCPi01    = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v40_CCPi0_run1_dlana_stripped_v2.root")    
    

inExtFF1    = inExt1.Get("dlana/FinalVertexVariables")
inOvrFF1    = inOvr1.Get("FinalVertexVariables")
inNueFF1    = inNue1.Get("FinalVertexVariables")
inNueLowFF1 = inNueLow1.Get("dlana/FinalVertexVariables")
inNumuLowFF1= inNumuLow1.Get("dlana/FinalVertexVariables")
inCCPi0FF1    = inCCPi01.Get("dlana/FinalVertexVariables")
inNCPi0FF1    = inNCPi01.Get("dlana/FinalVertexVariables")

#inDirtFF   = inDirt.Get("dlana/FinalVertexVariables") 



# ---------------- Run 2 -------------- #

inOvr2      = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/run2_overlay_stripmine.root")
inNue2      = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v29e_dl_run2_bnb_intrinsics_nue_overlay_finalbdt.root")
inNueLow2   = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_intrinsic_nue_lowe_run2_fvv_stripped.root")
inNumuLow2  = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_bnb_overlay_lowe_run2_fvv_stripped.root")
    

inOvrFF2    = inOvr2.Get("FinalVertexVariables")
inNueFF2    = inNue2.Get("dlana/FinalVertexVariables")
inNueLowFF2 = inNueLow2.Get("dlana/FinalVertexVariables")
inNumuLowFF2= inNumuLow2.Get("dlana/FinalVertexVariables")




# ---------------- Run 3 -------------- #

inExt3      = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v29e_dl_run3_G1_extbnb_finalbdt.root")
inDirt3      = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v29e_dl_run3b_bnb_dirt_overlay.root")
inOvr3      = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v29e_dl_run3b_bnb_nu_overlay_nocrtremerge_finalbdt.root")
inNue3      = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v29e_dl_run3b_bnb_intrinsic_nue_overlay_nocrtremerge_finalbdt.root")
inNueLow3   = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v29e_dl_run3b_intrinsic_nue_LowE.root")
inNumuLow3  = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v29e_run3b_bnb_nu_overlay_LowE.root")
inNCPi03    = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v40_NCPi0_run3_dlana_stripped_v2.root")
inCCPi03    = TFile.Open("/home/nwkamp/Research/MicroBooNE/Data/mcc9_v40_CCPi0_run3_dlana_stripped_v2.root")
    
           
inExtFF3    = inExt3.Get("dlana/FinalVertexVariables")
inDirtFF3   = inDirt3.Get("dlana/FinalVertexVariables")
inOvrFF3    = inOvr3.Get("dlana/FinalVertexVariables")
inNueFF3    = inNue3.Get("dlana/FinalVertexVariables")
inNueLowFF3 = inNueLow3.Get("dlana/FinalVertexVariables")
inNumuLowFF3 = inNumuLow3.Get("dlana/FinalVertexVariables")
inCCPi0FF3    = inCCPi03.Get("dlana/FinalVertexVariables")
inNCPi0FF3    = inNCPi03.Get("dlana/FinalVertexVariables")

Warning in <TClass::Init>: no dictionary for class larlite::event_gtruth is available
Warning in <TClass::Init>: no dictionary for class larlite::gtruth is available
Warning in <TClass::Init>: no dictionary for class larlite::event_mctruth is available
Warning in <TClass::Init>: no dictionary for class larlite::mctruth is available
Warning in <TClass::Init>: no dictionary for class larlite::mcnu is available
Warning in <TClass::Init>: no dictionary for class larlite::mcpart is available
Warning in <TClass::Init>: no dictionary for class larlite::mctrajectory is available
Warning in <TClass::Init>: no dictionary for class larlite::event_mcflux is available
Warning in <TClass::Init>: no dictionary for class larlite::mcflux is available
Warning in <TClass::Init>: no dictionary for class larlite::event_mcshower is available
Warning in <TClass::Init>: no dictionary for class larlite::mcshower is available
Warning in <TClass::Init>: no dictionary for class larlite::mcstep is available
Warnin

In [3]:
GENIE_tune = 2
apply_pi0_weights = True

weightdir = '../aux/'
inference = 'avgscore'
date = 'Mar24'
pi0filename = {0:'final',5:'final',1:'Kincut',2:'highE',3:'lowBDT'}[cutMode] + '_' + inference
pi0filename2 = {0:'final',5:'final',1:'loosecuts',2:'highE',3:'lowBDT'}[cutMode] + '_' + inference

#date = 'May25_lineweights'
#pi0filename = 'final_avgscore'
#pi0filename2 = 'test'


pi0w_numu_1 = open(weightdir+"1e1p_pi0weights/"+pi0filename+"/"+date+"_pi0_weights_1e1p_"+pi0filename2+"_numu_run1.txt","r")
pi0w_nue_1 = open(weightdir+"1e1p_pi0weights/"+pi0filename+"/"+date+"_pi0_weights_1e1p_"+pi0filename2+"_nue_run1.txt","r")
pi0w_nue_low_1 = None
pi0w_numu_low_1 = None
pi0w_ncpi0_1 = open(weightdir+"1e1p_pi0weights/"+pi0filename+"/"+date+"_pi0_weights_1e1p_"+pi0filename2+"_ncpi0_run1.txt","r")
pi0w_ccpi0_1 = open(weightdir+"1e1p_pi0weights/"+pi0filename+"/"+date+"_pi0_weights_1e1p_"+pi0filename2+"_ccpi0_run1.txt","r")

pi0w_numu_2 = open(weightdir+"1e1p_pi0weights/"+pi0filename+"/"+date+"_pi0_weights_1e1p_"+pi0filename2+"_numu_run2.txt","r")
pi0w_nue_2 = open(weightdir+"1e1p_pi0weights/"+pi0filename+"/"+date+"_pi0_weights_1e1p_"+pi0filename2+"_nue_run2.txt","r")

pi0w_numu_3 = open(weightdir+"1e1p_pi0weights/"+pi0filename+"/"+date+"_pi0_weights_1e1p_"+pi0filename2+"_numu_run3.txt","r")
pi0w_nue_3 = open(weightdir+"1e1p_pi0weights/"+pi0filename+"/"+date+"_pi0_weights_1e1p_"+pi0filename2+"_nue_run3.txt","r")
pi0w_nue_low_3 = None
pi0w_numu_low_3 = None
pi0w_ncpi0_3 = open(weightdir+"1e1p_pi0weights/"+pi0filename+"/"+date+"_pi0_weights_1e1p_"+pi0filename2+"_ncpi0_run3.txt","r")
pi0w_ccpi0_3 = open(weightdir+"1e1p_pi0weights/"+pi0filename+"/"+date+"_pi0_weights_1e1p_"+pi0filename2+"_ccpi0_run3.txt","r")


    
if GENIE_tune==1:
    ovrCVFile1  = TFile.Open(weightdir+"weights_forCV_v40_bnb_nu_run1.root") 
    nueCVFile1  = TFile.Open(weightdir+"weights_forCV_v40_intrinsic_nue_run1.root") 
    nueLowCVFile1 = TFile.Open(weightdir+"weights_forCV_v40_intrinsic_nue_lowE_run1.root")
    numuLowCVFile1 = TFile.Open(weightdir+"weights_forCV_v40_bnb_nu_lowE_run1.root")
    ovrCVFile2  = TFile.Open(weightdir+"weights_forCV_v40_bnb_nu_run2.root") 
    nueCVFile2  = TFile.Open(weightdir+"weights_forCV_v40_intrinsic_nue_run2.root")
    ovrCVFile3  = TFile.Open(weightdir+"weights_forCV_v40_bnb_nu_run3.root") 
    nueCVFile3  = TFile.Open(weightdir+"weights_forCV_v40_intrinsic_nue_run3.root") 
    nueLowCVFile3 = TFile.Open(weightdir+"weights_forCV_v40_intrinsic_nue_lowE_run3.root")
    numuLowCVFile3 = TFile.Open(weightdir+"weights_forCV_v40_bnb_nu_lowE_run3.root")    
elif GENIE_tune==2:
    ovrCVFile1  = TFile.Open(weightdir+"weights_forCV_v48_Sep24_bnb_nu_run1.root") 
    nueCVFile1  = TFile.Open(weightdir+"weights_forCV_v48_Sep24_intrinsic_nue_run1.root") 
    nueLowCVFile1 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_intrinsic_nue_lowE_run1.root")
    numuLowCVFile1 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_bnb_nu_lowE_run1.root")
    ovrCVFile2  = TFile.Open(weightdir+"weights_forCV_v48_Sep24_bnb_nu_run2.root") 
    nueCVFile2  = TFile.Open(weightdir+"weights_forCV_v48_Sep24_intrinsic_nue_run2.root")
    ovrCVFile3  = TFile.Open(weightdir+"weights_forCV_v48_Sep24_bnb_nu_run3.root") 
    nueCVFile3  = TFile.Open(weightdir+"weights_forCV_v48_Sep24_intrinsic_nue_run3.root") 
    nueLowCVFile3 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_intrinsic_nue_lowE_run3.root")
    numuLowCVFile3 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_bnb_nu_lowE_run3.root")
    
nueLowCVFile2 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_intrinsic_nue_lowE_run2.root")
numuLowCVFile2 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_bnb_nu_lowE_run2.root")
NCPi0CVFile1 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_nc_pi0_run1.root")
NCPi0CVFile3 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_nc_pi0_run3.root")
CCPi0CVFile1 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_cc_pi0_run1.root")
CCPi0CVFile3 = TFile.Open(weightdir+"weights_forCV_v48_Sep24_cc_pi0_run3.root")
    
    
ovrCV1      = ovrCVFile1.Get("eventweight_tree")
nueCV1      = nueCVFile1.Get("eventweight_tree")
nueLowCV1   = nueLowCVFile1.Get("eventweight_tree")
numuLowCV1  = nueLowCVFile1.Get("eventweight_tree")
ovrCV2      = ovrCVFile2.Get("eventweight_tree")
nueCV2      = nueCVFile2.Get("eventweight_tree")
nueLowCV2   = nueLowCVFile2.Get("eventweight_tree")
numuLowCV2  = nueLowCVFile2.Get("eventweight_tree")
ovrCV3      = ovrCVFile3.Get("eventweight_tree")
nueCV3      = nueCVFile3.Get("eventweight_tree")
nueLowCV3   = nueLowCVFile3.Get("eventweight_tree")
numuLowCV3  = numuLowCVFile3.Get("eventweight_tree")

ccpi0_1     = CCPi0CVFile1.Get("eventweight_tree")
ccpi0_3     = CCPi0CVFile3.Get("eventweight_tree")
ncpi0_1     = NCPi0CVFile1.Get("eventweight_tree")
ncpi0_3     = NCPi0CVFile3.Get("eventweight_tree")

ovrCVDict = {}
nueCVDict = {}
nueLowCVDict = {}
numuLowCVDict = {}
leeCVDict = {}
leeLowCVDict = {}

ncpi0CVDict = {}
ccpi0CVDict = {}




# print('run 1 ccpi0')
# for x in ccpi0_1:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight = x.xsec_corr_weight
#     if isnan(weight) or isinf(weight):
#         ccpi0CVDict[idx] = 1.0
#     else:
#         ccpi0CVDict[idx] = weight
        
# if apply_pi0_weights and pi0w_ccpi0_1 is not None:
#     all_data = [(line.strip()).split(',') for line in pi0w_ccpi0_1.readlines()]
#     data = all_data[:]
#     for line in data:
#         if line[0]=='run': continue
#         r = int(line[0])
#         s = int(line[1])
#         e = int(line[2])
#         E = round(float(line[3]),0)
#         w = float(line[-1])
#         if w>0:
#             for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
#                 if idx in ccpi0CVDict: 
#                     ccpi0CVDict[idx]*=w
#                     continue
#     pi0w_ccpi0_1.close()
        
# print('run 3 ccpi0')
# for x in ccpi0_3:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight = x.xsec_corr_weight
#     if isnan(weight) or isinf(weight):
#         ccpi0CVDict[idx] = 1.0
#     else:
#         ccpi0CVDict[idx] = weight
        
# if apply_pi0_weights and pi0w_ccpi0_3 is not None:
#     all_data = [(line.strip()).split(',') for line in pi0w_ccpi0_3.readlines()]
#     data = all_data[:]
#     for line in data:
#         if line[0]=='run': continue
#         r = int(line[0])
#         s = int(line[1])
#         e = int(line[2])
#         E = round(float(line[3]),0)
#         w = float(line[-1])
#         if w>0:
#             for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
#                 if idx in ccpi0CVDict: 
#                     ccpi0CVDict[idx]*=w
#                     continue
#     pi0w_ccpi0_3.close()
        
# print('run 1 ncpi0')
# for x in ncpi0_1:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight = x.xsec_corr_weight
#     if isnan(weight) or isinf(weight):
#         ncpi0CVDict[idx] = 1.0
#     else:
#         ncpi0CVDict[idx] = weight
        
# if apply_pi0_weights and pi0w_ncpi0_1 is not None:
#     all_data = [(line.strip()).split(',') for line in pi0w_ncpi0_1.readlines()]
#     data = all_data[:]
#     for line in data:
#         if line[0]=='run': continue
#         r = int(line[0])
#         s = int(line[1])
#         e = int(line[2])
#         E = round(float(line[3]),0)
#         w = float(line[-1])
#         if w>0:
#             for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
#                 if idx in ncpi0CVDict: 
#                     ncpi0CVDict[idx]*=w
#                     continue
#     pi0w_ncpi0_1.close()
        
# print('run 3 ncpi0')
# for x in ncpi0_3:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight = x.xsec_corr_weight
#     if isnan(weight) or isinf(weight):
#         ncpi0CVDict[idx] = 1.0
#     else:
#         ncpi0CVDict[idx] = weight
        
# if apply_pi0_weights and pi0w_ncpi0_3 is not None:
#     all_data = [(line.strip()).split(',') for line in pi0w_ncpi0_3.readlines()]
#     data = all_data[:]
#     for line in data:
#         if line[0]=='run': continue
#         r = int(line[0])
#         s = int(line[1])
#         e = int(line[2])
#         E = round(float(line[3]),0)
#         w = float(line[-1])
#         if w>0:
#             for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
#                 if idx in ncpi0CVDict: 
#                     ncpi0CVDict[idx]*=w
#                     continue
            
#     pi0w_ncpi0_3.close()
    


# print('run 1 bnb nu')
# for x in ovrCV1:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight = x.xsec_corr_weight
#     if isnan(weight) or isinf(weight):
#         ovrCVDict[idx] = 1.0
#     else:
#         ovrCVDict[idx] = weight

# if apply_pi0_weights:
#     all_data = [(line.strip()).split(',') for line in pi0w_numu_1.readlines()]
#     data = all_data[:]
#     for line in data:
#         if line[0]=='run': continue
#         r = int(line[0])
#         s = int(line[1])
#         e = int(line[2])
#         E = round(float(line[3]),0)
#         w = float(line[-1])
#         if w>0:
#             for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
#                 if idx in ovrCVDict: 
#                     ovrCVDict[idx]*=w
#                     continue
#     pi0w_numu_1.close()
    
        
# print('run 1 bnb nu lowE')
# for x in numuLowCV1:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight = x.xsec_corr_weight
#     if isnan(weight) or isinf(weight):
#         numuLowCVDict[idx] = 1.0
#     else:
#         numuLowCVDict[idx] = weight

# if apply_pi0_weights:
#     if pi0w_numu_low_1 is not None:
#         all_data = [(line.strip()).split(',') for line in pi0w_numu_low_1.readlines()]
#         data = all_data[:]
#         for line in data:
#             if line[0]=='run': continue
#             r = int(line[0])
#             s = int(line[1])
#             e = int(line[2])
#             E = round(float(line[3]),0)
#             w = float(line[-1])
#             if w>0:
#                 for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
#                     if idx in numuLowCVDict: 
#                         numuLowCVDict[idx]*=w
#                         continue 
#         pi0w_numu_low_1.close()

# print('run 1 intrinsic nue')
# for x in nueCV1:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight    = x.xsec_corr_weight
#     leeweight = x.lee_weight
    
#     if isnan(weight) or isinf(weight):
#         nueCVDict[idx] = 1.0
#     else:
#         nueCVDict[idx] = weight

#     if isnan(leeweight) or isinf(leeweight):
#         leeCVDict[idx] = 0.0
#     else:
#         leeCVDict[idx] = leeweight


# if apply_pi0_weights:
#     all_data = [(line.strip()).split(',') for line in pi0w_nue_1.readlines()]
#     data = all_data[:]
#     for line in data:
#         if line[0]=='run': continue
#         r = int(line[0])
#         s = int(line[1])
#         e = int(line[2])
#         E = round(float(line[3]),0)
#         w = float(line[-1])
#         if w>0:
#             for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
#                 if idx in nueCVDict: 
#                     nueCVDict[idx]*=w
#                     leeCVDict[idx]*=w
#                     continue
#     pi0w_nue_1.close()

        
# print('run 1 intrinsic nue lowE')
# for x in nueLowCV1:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight    = x.xsec_corr_weight
#     leeweight = x.lee_weight
    
#     if isnan(weight) or isinf(weight):
#         nueLowCVDict[idx] = 1.0
#     else:
#         nueLowCVDict[idx] = weight

#     if isnan(leeweight) or isinf(leeweight):
#         leeLowCVDict[idx] = 0.0
#     else:
#         leeLowCVDict[idx] = leeweight


# if apply_pi0_weights and pi0w_nue_low_1 is not None:
#     all_data = [(line.strip()).split(',') for line in pi0w_nue_low_1.readlines()]
#     data = all_data[:]
#     for line in data:
#         if line[0]=='run': continue
#         r = int(line[0])
#         s = int(line[1])
#         e = int(line[2])
#         E = round(float(line[3]),0)
#         w = float(line[-1])
#         if w>0:
#             for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
#                 if idx in nueLowCVDict: 
#                     nueLowCVDict[idx]*=w
#                     leeLowCVDict[idx]*=w
#                     continue
            
#     pi0w_nue_low_1.close()

        
# # ---------------- Run 2 -------------- #

# print('run 2 bnb nu')
# for x in ovrCV2:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight = x.xsec_corr_weight
#     if isnan(weight) or isinf(weight):
#         ovrCVDict[idx] = 1.0
#     else:
#         ovrCVDict[idx] = weight
        
# if apply_pi0_weights:
#     all_data = [(line.strip()).split(',') for line in pi0w_numu_2.readlines()]
#     data = all_data[:]
#     for line in data:
#         if line[0]=='run': continue
#         r = int(line[0])
#         s = int(line[1])
#         e = int(line[2])
#         E = round(float(line[3]),0)
#         w = float(line[-1])
#         if w>0:
#             for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
#                 if idx in ovrCVDict: 
#                     ovrCVDict[idx]*=w
#                     continue
#     pi0w_numu_2.close()
    
# print('run 2 bnb nu lowE')
# for x in numuLowCV2:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight = x.xsec_corr_weight
#     if isnan(weight) or isinf(weight):
#         numuLowCVDict[idx] = 1.0
#     else:
#         numuLowCVDict[idx] = weight

# print('run 2 intrinsic nue')
# for x in nueCV2:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight    = x.xsec_corr_weight
#     leeweight = x.lee_weight
    
#     if isnan(weight) or isinf(weight):
#         nueCVDict[idx] = 1.0
#     else:
#         nueCVDict[idx] = weight

#     if isnan(leeweight) or isinf(leeweight):
#         leeCVDict[idx] = 0.0
#     else:
#         leeCVDict[idx] = leeweight


# if apply_pi0_weights:
#     all_data = [(line.strip()).split(',') for line in pi0w_nue_2.readlines()]
#     data = all_data[:]
#     for line in data:
#         if line[0]=='run': continue
#         r = int(line[0])
#         s = int(line[1])
#         e = int(line[2])
#         E = round(float(line[3]),0)
#         w = float(line[-1])
#         if w>0:
#             for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
#                 if idx in nueCVDict: 
#                     nueCVDict[idx]*=w
#                     leeCVDict[idx]*=w
#                     continue
#     pi0w_nue_2.close()
    
# print('run 2 intrinsic nue lowE')
# for x in nueLowCV2:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight    = x.xsec_corr_weight
#     leeweight = x.lee_weight
    
#     if isnan(weight) or isinf(weight):
#         nueLowCVDict[idx] = 1.0
#     else:
#         nueLowCVDict[idx] = weight

#     if isnan(leeweight) or isinf(leeweight):
#         leeLowCVDict[idx] = 0.0
#     else:
#         leeLowCVDict[idx] = leeweight



# # ---------------- Run 3 -------------- #

# print('run 3 bnb nu')
# for x in ovrCV3:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight = x.xsec_corr_weight
#     if isnan(weight) or isinf(weight):
#         ovrCVDict[idx] = 1.0
#     else:
#         ovrCVDict[idx] = weight
        
# if apply_pi0_weights:
#     all_data = [(line.strip()).split(',') for line in pi0w_numu_3.readlines()]
#     data = all_data[:]
#     for line in data:
#         if line[0]=='run': continue
#         r = int(line[0])
#         s = int(line[1])
#         e = int(line[2])
#         E = round(float(line[3]),0)
#         w = float(line[-1])
#         if w>0:
#             for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
#                 if idx in ovrCVDict: 
#                     ovrCVDict[idx]*=w
#                     continue
#     pi0w_numu_3.close()
        
# print('run 3 bnb nu lowE')
# for x in numuLowCV3:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight = x.xsec_corr_weight
#     if isnan(weight) or isinf(weight):
#         numuLowCVDict[idx] = 1.0
#     else:
#         numuLowCVDict[idx] = weight

# if apply_pi0_weights:
#     if pi0w_numu_low_3 is not None:
#         all_data = [(line.strip()).split(',') for line in pi0w_numu_low_3.readlines()]
#         data = all_data[:]
#         for line in data:
#             if line[0]=='run': continue
#             r = int(line[0])
#             s = int(line[1])
#             e = int(line[2])
#             E = round(float(line[3]),0)
#             w = float(line[-1])
#             if w>0:
#                 for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
#                     if idx in numuLowCVDict: 
#                         numuLowCVDict[idx]*=w
#                         continue
#         pi0w_numu_low_3.close()
        
# print('run 3 intrinsic nue')
# for x in nueCV3:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight    = x.xsec_corr_weight
#     leeweight = x.lee_weight
    
#     if isnan(weight) or isinf(weight):
#         nueCVDict[idx] = 1.0
#     else:
#         nueCVDict[idx] = weight

#     if isnan(leeweight) or isinf(leeweight):
#         leeCVDict[idx] = 0.0
#     else:
#         leeCVDict[idx] = leeweight


# if apply_pi0_weights:
#     all_data = [(line.strip()).split(',') for line in pi0w_nue_3.readlines()]
#     data = all_data[:]
#     for line in data:
#         if line[0]=='run': continue
#         r = int(line[0])
#         s = int(line[1])
#         e = int(line[2])
#         E = round(float(line[3]),0)
#         w = float(line[-1])
#         if w>0:
#             for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
#                 if idx in nueCVDict: 
#                     nueCVDict[idx]*=w
#                     leeCVDict[idx]*=w
#                     continue
#     pi0w_nue_3.close()


# print('run 3 intrinsic nue lowE')
# for x in nueLowCV3:
#     idx = tuple((x.run,x.subrun,x.event,round(x.nu_energy_true,0)))
#     weight    = x.xsec_corr_weight
#     leeweight = x.lee_weight
    
#     if isnan(weight) or isinf(weight):
#         nueLowCVDict[idx] = 1.0
#     else:
#         nueLowCVDict[idx] = weight

#     if isnan(leeweight) or isinf(leeweight):
#         leeLowCVDict[idx] = 0.0
#     else:
#         leeLowCVDict[idx] = leeweight


# if apply_pi0_weights and pi0w_nue_low_3 is not None:
#     all_data = [(line.strip()).split(',') for line in pi0w_nue_low_3.readlines()]
#     data = all_data[:]
#     for line in data:
#         if line[0]=='run': continue
#         r = int(line[0])
#         s = int(line[1])
#         e = int(line[2])
#         E = round(float(line[3]),0)
#         w = float(line[-1])
#         if w>0:
#             for idx in [tuple((r,s,e,E)),tuple((r,s,e,E-1)),tuple((r,s,e,E+1))]:
#                 if idx in nueLowCVDict: 
#                     nueLowCVDict[idx]*=w
#                     leeLowCVDict[idx]*=w
#                     continue
#     pi0w_nue_low_3.close()
        
        



In [4]:

#tag='7July2021_vA_fullLowE_withPi0Sample_newShowerCalib_noEnu'
tag='3March2021_vA_fullLowE_withPi0Sample_newShowerCalib'
BDTnumlist=range(20)
oldBDT = False

ensemble = BDTensemble(tag,BDTnumlist,oldBDT=oldBDT)


SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R1_0_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R1_0_overlay_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R1_1_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328

SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R1_14_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R1_14_overlay_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R1_15_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:

SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R2_3_ext.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R2_3_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R2_3_overlay_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarnin

SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R2_8_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R2_8_overlay_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R2_8_ccpi0.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarn

SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R2_13_ccpi0.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R2_13_ncpi0.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R2_14_ext.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Em

SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R2_19_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R2_19_overlay_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R2_19_ccpi0.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserW

SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R3_11_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R3_11_overlay_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:328: UserWarning: loadtxt: Empty input file: "/home/nwkamp/Research/MicroBooNE/1L1PSelection/1e1pBDT/BDTWeights/new/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/ValSample_R3_12_intrinsics_lowE.txt"
  self.valrse[r][b][filetag] = np.loadtxt(bdtsavedir+'ValSample_R%i_%i_%s.txt'%(r,b,filetag),dtype=int)
SelectionHelper.py:

In [5]:
### All possible cuts on data/mc ###
lowEpatch      = False#True if fset in [0,5,8,9] else False

def printRSE(tree,fname):
    f = open(fname,'w')
    for x in tree:
        print(x.run,x.subrun,x.event,file=f)
    f.close()
    

In [6]:
filetaglist = {1:['data','ext','overlay','overlay_lowE','intrinsics','intrinsics_lowE','ncpi0','ccpi0'],
               2:['data','overlay','overlay_lowE','intrinsics','intrinsics_lowE'],
               3:['data','dirt','ext','overlay','overlay_lowE','intrinsics','intrinsics_lowE','ncpi0','ccpi0']}

tlist = {1:[inDatFF1,inExtFF1,inOvrFF1,inNumuLowFF1,inNueFF1,inNueLowFF1,inNCPi0FF1,inCCPi0FF1],
         2:[inDatFF2,inOvrFF2,inNumuLowFF2,inNueFF2,inNueLowFF2],
         3:[inDatFF3,inDirtFF3,inExtFF3,inOvrFF3,inNumuLowFF3,inNueFF3,inNueLowFF3,inNCPi0FF3,inCCPi0FF3]}

GENIElist = {1:[None,None,ovrCVDict,numuLowCVDict,nueCVDict,nueLowCVDict,ncpi0CVDict,ccpi0CVDict],
             2:[None,ovrCVDict,numuLowCVDict,nueCVDict,nueLowCVDict],
             3:[None,None,None,ovrCVDict,numuLowCVDict,nueCVDict,nueLowCVDict,ncpi0CVDict,ccpi0CVDict]}

LEElist = {1:[None,None,None,None,leeCVDict,leeLowCVDict,None,None],
           2:[None,None,None,leeCVDict,leeLowCVDict],
           3:[None,None,None,None,None,leeCVDict,leeLowCVDict,None,None]}

psavedir = 'Parquets/'+tag+'/'
try:
    os.mkdir(psavedir)
except OSError as error:
    print(error)

for r in [1,2,3]:
    print('\n\n','#'*20,'RUN',r,'#'*20,'\n')

    for t,filetag,genieDict,leeDict in zip(tlist[r],
                                           filetaglist[r],
                                           GENIElist[r],
                                           LEElist[r]):
        if not lowEpatch and 'lowE' in filetag: continue
        if t is None: continue
        #printRSE(t,'TextFiles/ProcessedRSE/%s_run%i.txt'%(filetag,r))
        if 'data' in filetag: continue
        if r!=3 or filetag!='dirt': continue
        print('Sample:',filetag)
        if 'data' in filetag:
            df = selection(t,cutMode,filetag,r,lowEpatch,ensemble,POT=POTDICT[r])
            pstring = 'SelectionMode%i_%s_fset%i_run%i.parquet.gzip'%(cutMode,filetag,fset,r)
        else:
            df = selection(t,cutMode,filetag,r,lowEpatch,ensemble,genieDict=genieDict,leeDict=leeDict)
            pstring = 'SelectionMode%i_%s_fset%i_run%i.parquet.gzip'%(cutMode,filetag,fset,r)
        df.to_parquet(psavedir+pstring,compression='gzip')

[Errno 17] File exists: 'Parquets/3March2021_vA_fullLowE_withPi0Sample_newShowerCalib/'


 #################### RUN 1 #################### 



 #################### RUN 2 #################### 



 #################### RUN 3 #################### 

Sample: dirt
